# HW 6 

Your task is to create a Python program that reads data from the ASDN database and uses Pandas to compute, for each species in the database (for which there is egg data), the coefficient of variation of volume using the above formula. There are many ways this can be done. Because this assignment is primarily about programming in Python, please follow the steps below. Please submit your Python code when done.

## Step 1

Create a query that will return the distinct species for which there is egg data (not all species and not all nests have egg data), so that you can then loop over those species. Your query should return two columns, species code and scientific name. Please order the results in alphabetic order of scientific name.

In [36]:
import duckdb

In [38]:
# create a connection
conn = duckdb.connect("database.db")

# create a cursor
cur = conn.cursor()

### Psuedo-Code

- need to make query where nest_id in bird_eggs has egg_num as something
- then we need to go from nest_id in bird_eggs to nest_id in bird_nests 
- then we need to select species in bird_nests which is code in species
- and then keep scientific name in species 
- and order by scientific name 

In [39]:
# create query 
species_query = """SELECT DISTINCT Code, Scientific_name
    FROM Species LEFT JOIN Bird_nests ON Code = Species
    LEFT JOIN Bird_eggs ON Bird_nests.Nest_ID = Bird_eggs.Nest_ID
    WHERE Width IS NOT NULL AND Length IS NOT NULL
    ORDER BY Scientific_name ASC;"""

In [40]:
cur.execute(species_query)

cur.fetchall()

[('rutu', 'Arenaria interpres'),
 ('dunl', 'Calidris alpina'),
 ('wrsa', 'Calidris fuscicollis'),
 ('sepl', 'Charadrius semipalmatus'),
 ('reph', 'Phalaropus fulicarius'),
 ('amgp', 'Pluvialis dominica'),
 ('bbpl', 'Pluvialis squatarola')]

## Step 2

After you’ve connected to the database and created a cursor c, iterate over the species like so:

```species_query = """SELECT Code, Scientific_name FROM..."""
for row in c.execute(species_query).fetchall():  # DuckDB lame-o workaround
    species_code = row[0]
    scientific_name = row[1]
    # query egg data for that species (step 3)
    # compute statistics and print results (step 4)```

In [41]:
for row in cur.execute(species_query).fetchall():  # DuckDB lame-o workaround
    species_code = row[0]
    scientific_name = row[1]
    # query egg data for that species (step 3)
    # compute statistics and print results (step 4)

### Step 3

You will need to construct a query that gathers egg data for a given species, one species at a time; the species code will be a parameter to that query. You can compute the formula W^2L


in SQL or in Pandas. For simplicity, SQL is suggested:

egg_query = """SELECT Width*Width*Length AS Volume FROM..."""
Within the loop, you will want to execute the query on the current species in the loop iteration. You may use the Pandas function pd.read_sql to do so and also directly load the results into a dataframe:

df = pd.read_sql(egg_query, conn, ...)
Do a help(pd.read_sql) to figure out how to pass parameters to a query.

You may get a bunch of warnings from Pandas about how it “only supports SQLAlchemy…”. Just ignore them. (Sorry about that.)

In [42]:
egg_query = """SELECT (Width * Width * Length) AS Volume
    FROM Bird_eggs
    WHERE Nest_ID IN (
        SELECT Nest_ID 
        FROM Bird_nests 
        WHERE Species = ?
    );"""

for row in cur.execute(species_query).fetchall():  # DuckDB lame-o workaround
    species_code = row[0]
    scientific_name = row[1]
    cur2 = conn.cursor()
    cur2.execute(egg_query, (species_code,))
    print(f"species {scientific_name}, Volume: {cur2.fetchone()[0]}")
    # compute statistics and print results (step 4)

species Arenaria interpres, Volume: 18129.2578125
species Calidris alpina, Volume: 22976.552734375
species Calidris fuscicollis, Volume: 20778.173828125
species Charadrius semipalmatus, Volume: 16836.818359375
species Phalaropus fulicarius, Volume: 15369.4091796875
species Pluvialis dominica, Volume: 42623.4609375
species Pluvialis squatarola, Volume: 70019.4921875


## Step 4

Finally, and still within your loop, you’ll want to compute statistics and print out the results:

In [53]:
# load pandas
import pandas as pd

for row in cur.execute(species_query).fetchall():  # DuckDB lame-o workaround
    species_code = row[0]
    scientific_name = row[1]
    cur2 = conn.cursor()
    cur2.execute(egg_query, (species_code,))

    # Read SQL query into a pandas DataFrame
    df = pd.read_sql(egg_query, conn, params=(species_code,))
    
    # Compute statistics
    mean_volume = df['Volume'].mean()
    std_volume = df['Volume'].std()
    cv = round(std_volume / mean_volume * 100, 2)
    
    # Print results
    print(f"{scientific_name} {cv}%")



Arenaria interpres 21.12%
Calidris alpina 5.46%
Calidris fuscicollis 16.77%
Charadrius semipalmatus 8.99%
Phalaropus fulicarius 4.65%
Pluvialis dominica 19.88%
Pluvialis squatarola 6.94%


/var/folders/p3/by7gqrh17wx1v95b5kw3dl480000gn/T/ipykernel_5089/1753264129.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(egg_query, conn, params=(species_code,))
